# AIM : Download VoxCeleb1, extract audio samples and evaluate their quality to build the metadata csv

## Libraries

In [28]:
import os
import torch  
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from pesq import pesq
from pystoi import stoi
import librosa.core
import soundfile as sf
import torchaudio
import random
import shutil
from scipy.signal import butter, lfilter
import tempfile

# import the src folder
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
try:
    from quality_scores import butter_lowpass_filter, compute_snr, compute_psnr, compute_mse, compute_pesq, compute_stoi
except Exception as e:
    print("Failed to import quality_scores from src:", e)
    raise

# 1. load dataset

We are going to use a subset of LibriSpeech dataset available via torchaudio library.

In [29]:
# Use dev-clean (smallest subset) and handle the loading more carefully
DATA_DIR = "../data/librispeech_subset"
RAW_DIR = os.path.join(DATA_DIR, "raw")
os.makedirs(RAW_DIR, exist_ok=True)

try:
    # Download dev-clean (smallest subset, ~337MB)
    dataset = torchaudio.datasets.LIBRISPEECH(
        root="../data",
        url="dev-clean",  # smallest subset
        download=True
    )
    
    # Take a small random sample
    N_SAMPLES = 300
    total_files = len(dataset)
    sampled_indices = random.sample(range(total_files), min(N_SAMPLES, total_files))
    
    print(f"Saving {len(sampled_indices)} samples to {RAW_DIR}")
    for idx in tqdm(sampled_indices):
        try:
            # Load audio
            waveform, sample_rate, _, _, _, _ = dataset[idx]
            
            # Save as WAV
            filename = f"librispeech_sample_{idx:04d}.wav"
            filepath = os.path.join(RAW_DIR, filename)
            torchaudio.save(filepath, waveform, sample_rate)
            
        except Exception as e:
            print(f"Error processing file {idx}: {e}")
            continue
            
    print(f"Dataset prepared in {RAW_DIR}")
    
except Exception as e:
    print(f"Failed to load dataset: {e}")
    print("Please check your internet connection and try again.")

Saving 300 samples to ../data/librispeech_subset\raw


100%|██████████| 300/300 [00:05<00:00, 55.05it/s]

Dataset prepared in ../data/librispeech_subset\raw


# 2. Prepare audiofiles

In [30]:
AUDIO_DIR = RAW_DIR
files = librosa.util.find_files(AUDIO_DIR, ext=["wav"])

print(f"{len(files)} files found.")

300 files found.


# 3. Calculate audio quality metrics

In [31]:
MIN_DURATION = 2.0
TARGET_SR = 16000

results = []

for f in tqdm(files, desc="Computing metrics"):
    try:
        # Load audio
        y, sr = librosa.load(f, sr=TARGET_SR, mono=True)
        
        if len(y) < sr * MIN_DURATION:
            continue

        print(f"Processing file: {f}, duration: {len(y)/sr:.2f} sec")
        
        # Normalize
        y = y / (np.abs(y).max() + 1e-8)
        
        # Create slightly enhanced reference (low-pass filtered, less noisy)
        y_clean = butter_lowpass_filter(y, cutoff=8000, sr=TARGET_SR, order=5)
        print(f"Max before: {np.abs(y).max():.4f}, after low-pass: {np.abs(y_clean).max():.4f}")
        y_clean = y_clean / (np.abs(y_clean).max() + 1e-8)
        
        # Compute metrics comparing original to cleaned version
        snr = compute_snr(y_clean, y)
        psnr = compute_psnr(y_clean, y)
        mse = compute_mse(y_clean, y)
        pesq_score = compute_pesq(y_clean, y, TARGET_SR)
        stoi_score = compute_stoi(y_clean, y, TARGET_SR)
        
        # Estimate noise floor
        frame_length = 2048
        hop_length = 512
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)
        noise_floor = np.percentile(rms, 10)

        results.append({
            "file_path": f,
            "duration_sec": len(y) / TARGET_SR,
            "snr": snr,
            "psnr": psnr,
            "mse": mse,
            "pesq": pesq_score,
            "stoi": stoi_score,
            "noise_floor_db": 20 * np.log10(noise_floor + 1e-8)
        })
    except Exception as e:
        print(f"Error processing file {f}: {e}")
        continue

# Save results to CSV
df = pd.DataFrame(results)

# Display basic statistics
print("Dataset Statistics:")
print(df[["snr", "psnr", "mse", "pesq", "stoi"]].describe())

# Filter to keep higher quality samples
PESQ_MIN = 2.5 # PESQ ranges from -0.5 to 4.5
STOI_MIN = 0.85 # STOI ranges from 0 to 1
SNR_MIN = 15.0 # in dB, higher is better

df_filtered = df[
    (df["pesq"] >= PESQ_MIN) &
    (df["stoi"] >= STOI_MIN) &
    (df["snr"] >= SNR_MIN)
].reset_index(drop=True)
        

Computing metrics:   0%|          | 0/300 [00:00<?, ?it/s]c:\Users\leami\Documents\projet_git\GaussBuster\src\quality_scores.py:30: UserWarning: Normalized cutoff 1 out of (0,1). Clamped to 0.999999999.
  )


Processing file: c:\Users\leami\Documents\projet_git\GaussBuster\data\librispeech_subset\raw\librispeech_sample_0003.wav, duration: 9.90 sec
[quality_scores] butter_lowpass_filter called: cutoff=8000, sr=16000, order=5
[quality_scores] normalized cutoff (clamped) = 0.999999999
Max before: 1.0000, after low-pass: 1.0000


Computing metrics:   0%|          | 1/300 [00:00<01:43,  2.88it/s]

Processing file: c:\Users\leami\Documents\projet_git\GaussBuster\data\librispeech_subset\raw\librispeech_sample_0005.wav, duration: 9.01 sec
[quality_scores] butter_lowpass_filter called: cutoff=8000, sr=16000, order=5
[quality_scores] normalized cutoff (clamped) = 0.999999999
Max before: 1.0000, after low-pass: 1.0000


Computing metrics:   1%|          | 3/300 [00:00<01:05,  4.51it/s]

...

Processing file: c:\Users\leami\Documents\projet_git\GaussBuster\data\librispeech_subset\raw\librispeech_sample_2666.wav, duration: 10.10 sec
[quality_scores] butter_lowpass_filter called: cutoff=8000, sr=16000, order=5
[quality_scores] normalized cutoff (clamped) = 0.999999999
Max before: 1.0000, after low-pass: 1.0000


Computing metrics:  99%|█████████▉| 298/300 [01:05<00:00,  4.77it/s]

Processing file: c:\Users\leami\Documents\projet_git\GaussBuster\data\librispeech_subset\raw\librispeech_sample_2669.wav, duration: 5.97 sec
[quality_scores] butter_lowpass_filter called: cutoff=8000, sr=16000, order=5
[quality_scores] normalized cutoff (clamped) = 0.999999999
Max before: 1.0000, after low-pass: 1.0000
Processing file: c:\Users\leami\Documents\projet_git\GaussBuster\data\librispeech_subset\raw\librispeech_sample_2692.wav, duration: 7.84 sec
[quality_scores] butter_lowpass_filter called: cutoff=8000, sr=16000, order=5
[quality_scores] normalized cutoff (clamped) = 0.999999999
Max before: 1.0000, after low-pass: 1.0000


Computing metrics: 100%|██████████| 300/300 [01:05<00:00,  4.58it/s]

Dataset Statistics:
              snr        psnr           mse        pesq          stoi
count  293.000000  293.000000  2.930000e+02  293.000000  2.930000e+02
mean    81.786958  156.772804  2.352289e-15    4.643887  1.000000e+00
std      2.516309   11.039563  6.979035e-15    0.000027  2.373884e-16
min     74.743830  131.163982  7.475282e-19    4.643433  1.000000e+00
25%     79.950666  148.643481  3.746170e-17    4.643888  1.000000e+00
50%     81.747891  155.876444  2.584376e-16    4.643888  1.000000e+00
75%     83.572827  164.264125  1.366633e-15    4.643888  1.000000e+00
max     89.551435  181.263724  7.648950e-14    4.643888  1.000000e+00


In [32]:
df_filtered.head()

,file_path,duration_sec,snr,psnr,mse,pesq,stoi,noise_floor_db
0,c:\Users\leami\Documents\projet_git\GaussBuste...,9.900,81.189301,162.349944,5.821107e-17,4.643888,1.0,-45.822767
1,c:\Users\leami\Documents\projet_git\GaussBuste...,9.010,82.152657,146.742139,2.117318e-15,4.643888,1.0,-49.941374
2,c:\Users\leami\Documents\projet_git\GaussBuste...,5.120,84.259554,136.752935,2.112061e-14,4.643888,1.0,-57.223136
3,c:\Users\leami\Documents\projet_git\GaussBuste...,10.885,82.462309,134.171573,3.826861e-14,4.643888,1.0,-43.568558
4,c:\Users\leami\Documents\projet_git\GaussBuste...,4.225,84.055596,150.311826,9.307164e-16,4.643888,1.0,-40.646802


# 4. Save metadata CSV

In [33]:
print(f"High quality samples: {len(df_filtered)} samples kept.")

# Copy selected files to a new directory
AUDIO_DIR_HIGH_QUALITY = "../data/librispeech_subset/high_quality_audio"
metadata_path = os.path.join(DATA_DIR, "metadata.csv")

# Convert all paths to absolute paths
AUDIO_DIR_HIGH_QUALITY = os.path.abspath(AUDIO_DIR_HIGH_QUALITY)
DATA_DIR = os.path.abspath(DATA_DIR)

metadata_data = {
    "file": [],
    "duration_s": [],
    "snr": [],
    "psnr": [],
    "mse": [],
    "pesq": [],
    "stoi": [],
    "noise_floor_db": []
}

# Create directory if it doesn't exist
os.makedirs(AUDIO_DIR_HIGH_QUALITY, exist_ok=True)

for _, row in df_filtered.iterrows():
    try:
        # support both possible column names
        src_raw = row.get("file_path", row.get("file"))
        if not src_raw:
            print("Row missing file path, skipping row.")
            continue

        src_path = os.path.abspath(src_raw)

        if os.path.exists(src_path):
            filename = os.path.basename(src_path)
            dest_path = os.path.join(AUDIO_DIR_HIGH_QUALITY, filename)

            # Copy file
            shutil.copy2(src_path, dest_path)

            # Store only the filename in metadata, map duration and metrics with fallbacks
            metadata_data["file"].append(filename)
            metadata_data["duration_s"].append(row.get("duration_sec", row.get("duration_s", None)))
            metadata_data["pesq"].append(row.get("pesq", None))
            metadata_data["stoi"].append(row.get("stoi", None))
            metadata_data["snr"].append(row.get("snr", None))
            metadata_data["psnr"].append(row.get("psnr", None))
            metadata_data["mse"].append(row.get("mse", None))
            metadata_data["noise_floor_db"].append(row.get("noise_floor_db", None))
        else:
            print(f"Source file not found: {src_path}")
    except Exception as e:
        # avoid referring to src_path here in case it's not defined
        print(f"Error processing row: {str(e)}")
        continue

# Save the new metadata CSV
df_new_metadata = pd.DataFrame(metadata_data)
df_new_metadata.to_csv(metadata_path, index=False)
print(f"New metadata saved: {metadata_path}")

# Print summary
print(f"Successfully processed {len(metadata_data['file'])} files")
print(f"Files saved to: {AUDIO_DIR_HIGH_QUALITY}")
print(f"Metadata saved to: {metadata_path}")

High quality samples: 293 samples kept.
New metadata saved: ../data/librispeech_subset\metadata.csv
Successfully processed 293 files
Files saved to: c:\Users\leami\Documents\projet_git\GaussBuster\data\librispeech_subset\high_quality_audio
Metadata saved to: ../data/librispeech_subset\metadata.csv
